In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# TeamPJ - PUBG Finish Placement Prediction (Kernels Only)

#### ~ 2022 Jun 10 
Today To do list
- 모델링 1차 완료하기. 원하는 모델을 돌려보고, 처음 러프하게 돌려본 모델과의 차이와 이유를 분석하여 결론내기.

주말동안 할 것들
- 칼럼 : 본인이 담당한 칼럼에 대해서는 확실한 인사이트와 전처리 코드를 가져오기

- 캐글 : 레퍼런스 보고 인사이트 찾기. 자신의 담당칼럼 위주. 원하면 다른 칼럼에 대한 조사 또한 가능. 하지만 이 때, 본인의 칼럼이 부실하면 안됨.

- 전처리 데이터 파일 : 캐글에서 볼 때 전처리 파일은 팀에서 전처리한 CSV파일을 기반으로 진행. 캐글 레퍼런스와 결과값의 차이가 있을 때, 어떤 부분에서 차이가 있으며, 자신의 전처리코드를 어떻게 수정하면 좋을지에 대한 인사이트도 필요.

### Basic settings
- By reducing memory usage, we can prevent the situation of kernel issue

In [ ]:
#Basic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# For google colab memory issue
pd.set_option('mode.chained_assignment',  None)

In [ ]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    print('{')
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        
        col_type = str(df[col].dtype)
        if col_type == 'object':
            col_type = 'str'
        else:
            col_type = 'np.' + col_type    
        #print('\'' + col + '\':' + col_type + ',')
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    print('}')
    return df

In [ ]:
def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [ ]:
# Jupyter notebook
# train = import_data('./data/featured_train_2.csv')
# test = import_data('./data/test_V2.csv')
# pubg.head()

# Google Colab
print('PUBG train')
train = import_data('/content/drive/MyDrive/modlingPJ/input/featured_train_2.csv')
print('PUBG test')
test = import_data('/content/drive/MyDrive/modlingPJ/input/test_V2.csv')

PUBG train
Memory usage of dataframe is 610.70 MB
{
Memory usage after optimization is: 106.02 MB
Decreased by 82.6%
}
PUBG test
Memory usage of dataframe is 413.18 MB
{
Memory usage after optimization is: 121.74 MB
Decreased by 70.5%
}


In [ ]:
# Show every columns
pd.set_option('display.max_columns', 30)
pd.get_option("display.max_rows")
train.head(2)

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,kills,longestKill,revives,rideDistance,swimDistance,vehicleDestroys,walkDistance,weaponsAcquired,winPlacePerc,matchType2,cat_numGroups
0,0,0,0.0,0,0,0,60,0,0.0,0,0.000000,0.000000,0,244.75,1,0.444336,4,0
1,0,0,91.5,0,0,0,57,0,0.0,0,0.004501,11.039062,0,1434.00,5,0.640137,4,0


In [ ]:
pubg = train.copy()
pubg.head()

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,kills,longestKill,revives,rideDistance,swimDistance,vehicleDestroys,walkDistance,weaponsAcquired,winPlacePerc,matchType2,cat_numGroups
0,0,0,0.00000,0,0,0,60,0,0.00000,0,0.000000,0.000000,0,244.75,1,0.444336,4,0
1,0,0,91.50000,0,0,0,57,0,0.00000,0,0.004501,11.039062,0,1434.00,5,0.640137,4,0
2,1,0,68.00000,0,0,0,47,0,0.00000,0,0.000000,0.000000,0,161.75,2,0.775391,2,1
3,0,0,32.90625,0,0,0,75,0,0.00000,0,0.000000,0.000000,0,202.75,3,0.166748,4,0
4,0,0,100.00000,0,0,0,45,1,58.53125,0,0.000000,0.000000,0,49.75,2,0.187500,1,2


In [ ]:
pubg.cat_numGroups.value_counts()

0    2455328
1    1297627
2     694010
Name: cat_numGroups, dtype: int64

In [ ]:
# information of df
pubg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4446965 entries, 0 to 4446964
Data columns (total 18 columns):
 #   Column           Dtype  
---  ------           -----  
 0   assists          int8   
 1   boosts           int8   
 2   damageDealt      float16
 3   DBNOs            int8   
 4   headshotKills    int8   
 5   heals            int8   
 6   killPlace        int8   
 7   kills            int8   
 8   longestKill      float16
 9   revives          int8   
 10  rideDistance     float16
 11  swimDistance     float16
 12  vehicleDestroys  int8   
 13  walkDistance     float16
 14  weaponsAcquired  int16  
 15  winPlacePerc     float16
 16  matchType2       int8   
 17  cat_numGroups    int8   
dtypes: float16(6), int16(1), int8(11)
memory usage: 106.0 MB


#### Data fields - After preprocessing data

**== dropped ==**
- Id - Player’s Id => **drop**
- groupId - ID to identify a group within a match. If the same group of players plays in different matches, they will have a different groupId each time.  => **drop**
- matchId - ID to identify match. There are no matches that are in both the training and testing set.  => **drop**
- matchType - String identifying the game mode that the data comes from. The standard modes are “solo”, “duo”, “squad”, “solo-fpp”, “duo-fpp”, and “squad-fpp”; other modes are from events or custom matches.  => **drop**
- matchDuration - Duration of match in seconds.  => **drop**
- numGroups - Number of groups we have data for in the match.  => **drop**
- rankPoints - Elo-like ranking of player. This ranking is inconsistent and is being deprecated in the API’s next version, so use with caution. Value of -1 takes place of “None”.  => **drop**
- roadKills - Number of kills while in a vehicle.  => **drop**
- teamKills - Number of times this player killed a teammate.  => **drop**
- winPoints - Win-based external ranking of player. (Think of this as an Elo ranking where only winning matters.) If there is a value other than -1 in rankPoints, then any 0 in winPoints should be treated as a “None”.  => **drop**
- killPoints - Kills-based external ranking of player. (Think of this as an Elo ranking where only kills matter.) If there is a value other than -1 in rankPoints, then any 0 in killPoints should be treated as a “None”.  => **drop**
- maxPlace - Worst placement we have data for in the match. This may not match with numGroups, as sometimes the data skips over placements.  => **drop**


**== using ==**
- DBNOs - Number of enemy players knocked.
- assists - Number of enemy players this player damaged that were killed by teammates.
- boosts - Number of boost items used.
- damageDealt - Total damage dealt. Note: Self inflicted damage is subtracted.
- headshotKills - Number of enemy players killed with headshots.
- heals - Number of healing items used.
- killPlace - Ranking in match of number of enemy players killed.
- killStreaks - Max number of enemy players killed in a short amount of time.
- kills - Number of enemy players killed.
- longestKill - Longest distance between player and player killed at time of death. This may be misleading, as downing a player and driving away may lead to a large longestKill stat.
- revives - Number of times this player revived teammates.
- rideDistance - Total distance traveled in vehicles measured in meters.
- swimDistance - Total distance traveled by swimming measured in meters.
- vehicleDestroys - Number of vehicles destroyed.
- walkDistance - Total distance traveled on foot measured in meters.
- weaponsAcquired - Number of weapons picked up.
- winPlacePerc - The target of prediction. This is a percentile winning placement, where 1 corresponds to 1st place, and 0 corresponds to last place in the match. It is calculated off of maxPlace, not numGroups, so it is possible to have missing chunks in a match.


**== Added ==**
- matchType2 - simplify match type as 0: flare & crash, 1 : solo, 2 : duo, 4 : squad and drop matchType
- cat_numGroups - simplify nunmGroups as if nunmGroups < 40 : 0, if numGroups > 40 & numGroups < 80 : 1, numGroups > 80 : 2 and drop numGroups



In [ ]:
# Brief calculations
pubg.describe()

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,kills,longestKill,revives,rideDistance,swimDistance,vehicleDestroys,walkDistance,weaponsAcquired,winPlacePerc,matchType2,cat_numGroups
count,4.446965e+06,4.446965e+06,4446965.00,4.446965e+06,4.446965e+06,4.446965e+06,4.446965e+06,4.446965e+06,4.446965e+06,4.446965e+06,4.446965e+06,4446965.0,4.446965e+06,4446965.000,4.446965e+06,4.446965e+06,4.446965e+06,4.446965e+06
mean,2.338150e-01,1.106908e+00,NaN,6.578757e-01,2.268196e-01,1.370148e+00,4.759936e+01,9.247835e-01,NaN,1.646590e-01,NaN,NaN,7.918209e-03,NaN,3.660488e+00,NaN,2.913056e+00,6.039281e-01
std,5.885731e-01,1.715794e+00,NaN,1.145743e+00,6.021553e-01,2.679982e+00,2.746293e+01,1.558445e+00,NaN,4.721671e-01,NaN,NaN,9.261158e-02,NaN,2.456543e+00,0.000000e+00,1.223283e+00,7.425137e-01
min,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,155.125,2.000000e+00,1.999512e-01,2.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,84.25,0.000000e+00,0.000000e+00,0.000000e+00,4.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,685.500,3.000000e+00,4.582520e-01,4.000000e+00,0.000000e+00
75%,0.000000e+00,2.000000e+00,186.00,1.000000e+00,0.000000e+00,2.000000e+00,7.100000e+01,1.000000e+00,2.131250e+01,0.000000e+00,1.910400e-01,0.0,0.000000e+00,1976.000,5.000000e+00,7.407227e-01,4.000000e+00,1.000000e+00
max,2.200000e+01,3.300000e+01,6616.00,5.300000e+01,6.400000e+01,8.000000e+01,1.010000e+02,7.200000e+01,1.094000e+03,3.900000e+01,4.070400e+04,3824.0,5.000000e+00,25776.000,2.360000e+02,1.000000e+00,4.000000e+00,2.000000e+00


In [ ]:
# Drop missing value train, test
pubg = pubg.dropna(axis=0, how='any')
test = test.dropna(axis=0, how='any')

In [ ]:
pubg.isna().sum() # 0 checked

assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
kills              0
longestKill        0
revives            0
rideDistance       0
swimDistance       0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPlacePerc       0
matchType2         0
cat_numGroups      0
dtype: int64

In [ ]:
test.isna().sum() # 0 checked => dimenstion size different
# 'Id', 'groupId', 'matchId', 'killPoints', 'killStreaks', 'matchDuration', 'maxPlace', 
# 'matchType', 'numGroups', 'rankPoints', 'roadKills', 'teamKills' => needed to drop
# 'winPlacePerc' -> target
# matchType -> matchType2 needed
# numGroups -> cat_numGroups needed

Id                 0
groupId            0
matchId            0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killPoints         0
kills              0
killStreaks        0
longestKill        0
matchDuration      0
matchType          0
maxPlace           0
numGroups          0
rankPoints         0
revives            0
rideDistance       0
roadKills          0
swimDistance       0
teamKills          0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
dtype: int64

In [ ]:
# Standardize the size of dimension.
test = test.drop(columns=['Id', 'groupId', 'matchId', 'rankPoints', 'roadKills', 'matchDuration', 
'teamKills', 'killPoints', 'winPoints', 'maxPlace', 'killStreaks']) 
# killStreaks added together to drop columns

In [ ]:
## 윤상현
# matchType2 - test
test['matchType2'] = pd.NaT
maplist = {
    'squad' : 'squad',
    'duo' : 'duo',
    'solo' : 'solo',
    'crash' : 'event',
    'flare' : 'event'
}

for i in maplist:
  test.matchType2[(test['matchType'].str.contains(i)==True)&(test['matchType2'].isnull())]=maplist.get(i)


# solo ⮕ 1, duo ⮕ 2, squad ⮕ 4, falre & crash ⮕ 0 으로 전환
def convert(x):
    if x == "squad":
        return 4
    elif x == "duo":
        return 2
    elif x == "solo":
        return 1
    else : 
        return 0

test['matchType2'] = test['matchType2'].map(convert)


# test matchType drop
test.drop(['matchType'], axis=1, inplace=True)

In [ ]:
## 문석민
## numGroups column을 3구간으로 나눠서 categorical value로 표현.
## < 40, 40 < 80, 80 <

test['cat_numGroups'] = 0

test.loc[test['numGroups'] < 40, 'cat_numGroups'] = 0
test.loc[(test['numGroups'] > 40) & (test['numGroups'] < 80), 'cat_numGroups'] = 1
test.loc[test['numGroups'] > 80, 'cat_numGroups'] = 2

test = test.drop(columns='numGroups')

In [ ]:
test.columns

Index(['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals',
       'killPlace', 'kills', 'longestKill', 'revives', 'rideDistance',
       'swimDistance', 'vehicleDestroys', 'walkDistance', 'weaponsAcquired',
       'matchType2', 'cat_numGroups'],
      dtype='object')

In [ ]:
pubg.columns
# 'winPlacePerc' -> target checked & same size of dimensions checked.

Index(['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals',
       'killPlace', 'kills', 'longestKill', 'revives', 'rideDistance',
       'swimDistance', 'vehicleDestroys', 'walkDistance', 'weaponsAcquired',
       'winPlacePerc', 'matchType2', 'cat_numGroups'],
      dtype='object')

## Modeling


### Training
- train-test data split
- model training

#### Data Split

In [ ]:
# 학습을 위한 training / test dataset 나누기
from sklearn.model_selection import train_test_split

x = pubg.drop(columns='winPlacePerc')
y = pubg['winPlacePerc']

# 6 : 2 : 2 = train : validation : test
x_train, x_test, y_train, y_test = \
train_test_split(x, y, test_size=0.2, random_state=100)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((3557572, 17), (889393, 17), (3557572,), (889393,))

In [ ]:
# Feature Engineering
# fix : ValueError: Unknown label type: 'continuous' -> LogisticRegression
# LogisticRegression Recall that a logistic regression model requires 
# the values of the response variable to be categorical such as:
# 0 or 1 / “Yes” or “No” / “Pass” or “Fail”
#from sklearn import preprocessing
#from sklearn import utils

#convert y values to categorical values
#lab = preprocessing.LabelEncoder()
#y_transformed = lab.fit_transform(y_train)

#view transformed values
#print(y_transformed)
# ref : https://www.statology.org/valueerror-unknown-label-type-continuous/

##### Modeling 1st count

Regression Model list
- LinearRegression
- Lasso
- Ridge
- SVR
- KNeighborsRegressor
- DecisionTreeRegressor
- LGBMRegressor
- RandomForestRegressor
- XGBRegressor
- CatBoostRegressor

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
# Modeling test - > Trying same as before
from sklearn.linear_model import LinearRegression   # 1. Linear Regression
from sklearn.linear_model import Lasso              # 2. Lasso
from sklearn.linear_model import Ridge              # 3. Ridge
from xgboost.sklearn import XGBRegressor            # 4. XGBoost
from lightgbm.sklearn import LGBMRegressor          # 5. LightGBM

# S: SVR Package
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
# E: SVR Package

# added
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor
# from catboost import CatBoostRegressor # need to install in conda : conda install catboost


from sklearn.metrics import mean_absolute_error # mean_absolute_error -> mae


In [ ]:
## training
reg = LinearRegression()
reg2 = Lasso()
reg3 = Ridge()
reg4 = XGBRegressor()
reg5 = LGBMRegressor()

reg.fit(x_train, y_train)
reg2.fit(x_train, y_train)
reg3.fit(x_train, y_train)
reg4.fit(x_train, y_train)
reg5.fit(x_train, y_train)

pred_train = reg.predict(x_train)
pred_train2 = reg2.predict(x_train)
pred_train3 = reg3.predict(x_train)
pred_train4 = reg4.predict(x_train)
pred_train5 = reg5.predict(x_train)

pred_val = reg.predict(x_test)
pred_val2 = reg2.predict(x_test)
pred_val3 = reg3.predict(x_test)
pred_val4 = reg4.predict(x_test)
pred_val5 = reg5.predict(x_test)

mae_train = mean_absolute_error(y_train, pred_train)
mae_val = mean_absolute_error(y_test, pred_val)
mae_train2 = mean_absolute_error(y_train, pred_train2)
mae_val2 = mean_absolute_error(y_test, pred_val2)
mae_train3 = mean_absolute_error(y_train, pred_train3)
mae_val3 = mean_absolute_error(y_test, pred_val3)
mae_train4 = mean_absolute_error(y_train, pred_train4)
mae_val4 = mean_absolute_error(y_test, pred_val4)
mae_train5 = mean_absolute_error(y_train, pred_train5)
mae_val5 = mean_absolute_error(y_test, pred_val5)


print("1. Linear Regression\t, train=%.4f, val=%.4f" % (mae_train, mae_val))
print("2. Lasso\t\t, train=%.4f, val=%.4f" % (mae_train2, mae_val2))
print("3. Ridge\t\t, train=%.4f, val=%.4f" % (mae_train3, mae_val3))
print("4. XGBoost\t\t, train=%.4f, val=%.4f" % (mae_train4, mae_val4))
print("5. LightGBM\t\t, train=%.4f, val=%.4f" % (mae_train5, mae_val5))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.87965e-09): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


[00:00:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1. Linear Regression	, train=0.1050, val=0.1049
2. Lasso		, train=0.1264, val=0.1261
3. Ridge		, train=0.1050, val=0.1049
4. XGBoost		, train=0.0740, val=0.0741
5. LightGBM		, train=0.0681, val=0.0681


In [ ]:
# prediction!
result = reg.predict(x_test)
result2 = reg2.predict(x_test)
result3 = reg3.predict(x_test)
result4 = reg4.predict(x_test)
result5 = reg5.predict(x_test)

In [ ]:
# Summary!
print("---------- Linear Regression ---------")
print('MAE in training: %.4f' % mean_absolute_error(y_test, result))

print("---------- Lasso ---------")
print('MAE in training: %.4f' % mean_absolute_error(y_test, result2))

print("---------- Ridge ---------")
print('MAE in training: %.4f' % mean_absolute_error(y_test, result3))

print("---------- XGBoost ---------")
print('MAE in training: %.4f' % mean_absolute_error(y_test, result4))

print("---------- LightGBM ---------")
print('MAE in training: %.4f' % mean_absolute_error(y_test, result5))

---------- Linear Regression ---------
MAE in training: 0.1049
---------- Lasso ---------
MAE in training: 0.1261
---------- Ridge ---------
MAE in training: 0.1049
---------- XGBoost ---------
MAE in training: 0.0741
---------- LightGBM ---------
MAE in training: 0.0681


**Score from before data preprocessing 1st**


---------- Linear Regression ---------

MSE in training: 0.0926

---------- Lasso ---------

MSE in training: 0.1209

---------- Ridge ---------

MSE in training: 0.0926

---------- XGBoost ---------

MSE in training: 0.0694

---------- LightGBM ---------

MSE in training: 0.0603


#### Preprocessing data Again!
- killPlace 삭제 후 진행 -> 위의 이전 모델과 동일하게 돌렸을 때 오히려 성능 저하 확인 / 해당의 columns 역시 정답지나 마찬가지로서 적용되므로

In [ ]:
pubg.drop('killPlace', axis=1, inplace=True)
test.drop('killPlace', axis=1, inplace=True)

In [ ]:
# preprocessing data - check again
pubg.info() # 4446965 rows × 17 columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4446965 entries, 0 to 4446964
Data columns (total 17 columns):
 #   Column           Dtype  
---  ------           -----  
 0   assists          int8   
 1   boosts           int8   
 2   damageDealt      float16
 3   DBNOs            int8   
 4   headshotKills    int8   
 5   heals            int8   
 6   kills            int8   
 7   longestKill      float16
 8   revives          int8   
 9   rideDistance     float16
 10  swimDistance     float16
 11  vehicleDestroys  int8   
 12  walkDistance     float16
 13  weaponsAcquired  int16  
 14  winPlacePerc     float16
 15  matchType2       int8   
 16  cat_numGroups    int8   
dtypes: float16(6), int16(1), int8(10)
memory usage: 135.7 MB


In [ ]:
test.info() # 1934174 rows × 16 columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1934174 entries, 0 to 1934173
Data columns (total 16 columns):
 #   Column           Dtype  
---  ------           -----  
 0   assists          int8   
 1   boosts           int8   
 2   damageDealt      float16
 3   DBNOs            int8   
 4   headshotKills    int8   
 5   heals            int8   
 6   kills            int8   
 7   longestKill      float16
 8   revives          int8   
 9   rideDistance     float16
 10  swimDistance     float16
 11  vehicleDestroys  int8   
 12  walkDistance     float16
 13  weaponsAcquired  int16  
 14  matchType2       int64  
 15  cat_numGroups    int64  
dtypes: float16(5), int16(1), int64(2), int8(8)
memory usage: 81.2 MB


In [ ]:
# 학습을 위한 training / test dataset 나누기
from sklearn.model_selection import train_test_split

x = pubg.drop(columns='winPlacePerc')
y = pubg['winPlacePerc']

# 6 : 2 : 2 = train : validation : test
# x_train, x_test, y_train, y_test = \
# train_test_split(x, y, test_size=0.2, random_state=100)
x_train, x_test, y_train, y_test = \
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=100)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2668179, 17), (889393, 17), (2668179,), (889393,))

In [ ]:
# killPlcae 삭제 후 체크
# LinearRegression
from sklearn.linear_model import LinearRegression   # 1. Linear Regression
linReg = LinearRegression()
linReg.fit(x_train, y_train)
pred_train1 = linReg.predict(x_train)
pred_val1 = linReg.predict(x_test)
mae_train1 = mean_absolute_error(y_train, pred_train1)
mae_val1 = mean_absolute_error(y_test, pred_val1)
print('1. Linear Regression\t, train=%.4f, val=%.4f' % (mae_train1, mae_val1))
result1 = linReg.predict(x_test)
print('---------- Linear Regression ---------')
print('MAE in training: %.4f' % mean_absolute_error(y_test, result1))

1. Linear Regression	, train=0.1050, val=0.1051
---------- Linear Regression ---------
MAE in training: 0.1051


In [ ]:
# Lasso
from sklearn.linear_model import Lasso              # 2. Lasso

las = Lasso()
las.fit(x_train, y_train)
pred_train2 = las.predict(x_train)
pred_val2 = las.predict(x_test)
mae_train2 = mean_absolute_error(y_train, pred_train2)
mae_val2 = mean_absolute_error(y_test, pred_val2)
print('2. Lasso\t, train=%.4f, val=%.4f' % (mae_train2, mae_val2))
result2 = las.predict(x_test)
print('---------- Lasso ---------')
print('MAE in training: %.4f' % mean_absolute_error(y_test, result2))

2. Lasso	, train=0.1263, val=0.1265
---------- Lasso ---------
MAE in training: 0.1265


In [ ]:
from sklearn.linear_model import Ridge              # 3. Ridge

rid = Ridge()
rid.fit(x_train, y_train)
pred_train3 = rid.predict(x_train)
pred_val3 = rid.predict(x_test)
mae_train3 = mean_absolute_error(y_train, pred_train3)
mae_val3 = mean_absolute_error(y_test, pred_val3)
print('3. Ridge\t, train=%.4f, val=%.4f' % (mae_train3, mae_val3))
result3 = rid.predict(x_test)
print('---------- Ridge ---------')
print('MAE in training: %.4f' % mean_absolute_error(y_test, result3))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.89853e-09): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


3. Ridge	, train=0.1050, val=0.1051
---------- Ridge ---------
MAE in training: 0.1051


In [ ]:
from xgboost.sklearn import XGBRegressor            # 4. XGBoost

xgb = XGBRegressor()
xgb.fit(x_train, y_train)
pred_train4 = xgb.predict(x_train)
pred_val4 = xgb.predict(x_test)
mae_train4 = mean_absolute_error(y_train, pred_train4)
mae_val4 = mean_absolute_error(y_test, pred_val4)
print('4. XGBoost\t, train=%.4f, val=%.4f' % (mae_train4, mae_val4))
result4 = xgb.predict(x_test)
print('---------- XGBoost ---------')
print('MAE in training: %.4f' % mean_absolute_error(y_test, result4))

[00:08:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
4. XGBoost	, train=0.0742, val=0.0743
---------- XGBoost ---------
MAE in training: 0.0743


In [ ]:
from lightgbm.sklearn import LGBMRegressor          # 5. LightGBM

lgb = LGBMRegressor()
lgb.fit(x_train, y_train)
pred_train5 = lgb.predict(x_train)
pred_val5 = lgb.predict(x_test)
mae_train5 = mean_absolute_error(y_train, pred_train5)
mae_val5 = mean_absolute_error(y_test, pred_val5)
print('5. LightGBM\t, train=%.4f, val=%.4f' % (mae_train5, mae_val5))
result5 = lgb.predict(x_test)
print('---------- LightGBM ---------')
print('MAE in training: %.4f' % mean_absolute_error(y_test, result5))

5. LightGBM	, train=0.0681, val=0.0682
---------- LightGBM ---------
MAE in training: 0.0682


In [ ]:
# S: SVR Package 
from sklearn.svm import SVR          # 6. SVR Package
# from sklearn.pipeline import make_pipeline #regression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import GridSearchCV
# from sklearn.decomposition import PCA
# E: SVR Package

# svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
# svr_lin = SVR(kernel='linear', C=100, gamma='auto')
# svr_poly = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=0.1, coef0=1)

# svrs = [svr_rbf, svr_lin, svr_poly]
# kernel_label = ['RBF', 'Linear', 'Polynomial']
# model_color = ['m', 'c', 'g']

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_val = scaler.transform(x_val)
svr = make_pipeline(StandardScaler(), SVR(kernel='linear', C=1.0, epsilon=0.2))
svr.fit(x_train, y_train)
pipe = Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svr', SVR(epsilon=0.2))])

pipe.score(x_test, y_test)

# find the optimal values of C and gamma 
# svr = SVR()
# svr.fit(x_train, y_train)
# pred_train6 = svr.predict(x_train)
# pred_val6 = svr.predict(x_test)
# mae_train6 = mean_absolute_error(y_train, pred_train6)
# mae_val6 = mean_absolute_error(y_test, pred_val6)
# print('6. SVR Package\t, train=%.4f, val=%.4f' % (mae_train6, mae_val6))
# result6 = svr.predict(x_test)
# print('---------- SVR Package ---------')
# print('MAE in training: %.4f' % mean_absolute_error(y_test, result6))

In [ ]:
# SVR == Epsilon-Support Vector Regression.
from sklearn.decomposition import PCA

 svrmodel = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
 svrmodel = SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='scale', 
 kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
 svrmodel.fit(x_train, y_train)
 Pipeline(steps=[('standardscaler', StandardScaler()), ('svr', SVR(epsilon=0.2))])
 x_train.shape, y_train.shape, x_test.shape

pcr = make_pipeline(StandardScaler(), PCA(n_components=1), LinearRegression())
pcr.fit(x_train, y_train)
pca = pcr.named_steps['pca']  # retrieve the PCA step of the pipeline

fig, axes = plt.subplots(1, 2, figsize=(10, 3))
axes[0].scatter(pca.transform(x_test), y_test, alpha=0.3, label='ground truth')
axes[0].scatter(
    pca.transform(x_test), pcr.predict(x_test), alpha=0.3, label='predictions'
)
axes[0].set(
    xlabel='Projected data onto first PCA component', ylabel='y', title='PCR / svrmodel'
)
axes[0].legend()
axes[1].scatter(svrmodel.transform(x_test), y_test, alpha=0.3, label='ground truth')
axes[1].scatter(
    svrmodel.transform(x_test), svrmodel.predict(x_test), alpha=0.3, label='predictions'
)
axes[1].set(xlabel='Projected data onto first svrmodel component', ylabel='y', title='svrmodel')
axes[1].legend()
plt.tight_layout()
plt.show()

- SVR 하지 않은 이유 -> 너무 오래 걸림. 

In [ ]:
# Hyper-parameter tuning
# 1. Human Search
# 2. Grid Search(GridSearchCV) : 주어진 hp의 조합을 모두 돌려보는 방식.
# 3. Bayesian Optimization(hyperopt, optuna, ...) : hyper-parameter를 최적화하는 베이지안 방식을 사용.

# GridSearchCV
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#    "max_depth" : [3, 4, 5, 6, 7, -1], # 6
#    "learning_rate" : [0.1, 0.01, 0.05], # 3
#    "n_estimators" : [50, 100, 200] # 3
# } # 6x3x3 = 54

# gcv = GridSearchCV(reg5, param_grid, scoring='neg_mean_squared_error', verbose=1)
# gcv.fit(x_train, y_train)
#print(gcv.cv_results_)
# print(gcv.best_estimator_)
# print(gcv.best_params_)

In [ ]:
# 데이터에 대한 관점을 먼저 고민해보자.
# 나오는 데이터 대다수가 int, float로서 숫자인 타입들. => 양적자료&이산형 위주.
# 그 중에서 내 담당 
features = ['simpleMatchNum', 'damageDealt', 'boosts', 'heals', 'assists', 'revives', 'DBNOs']
# 해당의 features 을 조정할 방법 찾아보기.


In [ ]:
# test(Prediction)
# training set과 같은 전처리! V
# final_model = gcv.best_estimator_

In [ ]:
# # prediction!
# result = reg.predict(x_test)
# result2 = reg2.predict(x_test)
# result3 = reg3.predict(x_test)
# result4 = reg4.predict(x_test)
# result5 = reg5.predict(x_test)

In [ ]:
# # Summary!
# print("---------- Linear Regression ---------")
# print('MSE in training: %.4f' % mean_absolute_error(y_test, result))

# print("---------- Lasso ---------")
# print('MSE in training: %.4f' % mean_absolute_error(y_test, result2))

# print("---------- Ridge ---------")
# print('MSE in training: %.4f' % mean_absolute_error(y_test, result3))

# print("---------- XGBoost ---------")
# print('MSE in training: %.4f' % mean_absolute_error(y_test, result4))

# print("---------- LightGBM ---------")
# print('MSE in training: %.4f' % mean_absolute_error(y_test, result5))

In [ ]:
# SVR == Epsilon-Support Vector Regression.
#svrmodel = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
#svrmodel = SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='scale', 
#kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
#svrmodel.fit(x_train, y_train)
# Pipeline(steps=[('standardscaler', StandardScaler()), ('svr', SVR(epsilon=0.2))])
# x_train.shape, y_train.shape, x_test.shape
# code ref : https://python.hotexamples.com/examples/sklearn.svm/SVR/score/python-svr-score-method-examples.html

In [ ]:
# get_params(deep=True)
#y_pred_svr = svrmodel.predict(x_test)
#mean_absolute_error(y_test, y_pred_svr)
#svr_score = svrmodel.score(x_test, y_test)

#### Drop some columns and train again


In [ ]:
# # Drop colums train & test
# pubg_num = test.drop(['rankPoints', 'roadKills', 'matchDuration', 'teamKills'] ,axis=1)
# test_num = test.drop(['rankPoints', 'roadKills', 'matchDuration', 'teamKills'] ,axis=1)
# test_num.head()

In [ ]:
## training
reg = LinearRegression()
reg2 = Lasso()
reg3 = Ridge()
reg4 = XGBRegressor()
reg5 = LGBMRegressor()

reg.fit(x_train, y_train)
reg2.fit(x_train, y_train)
reg3.fit(x_train, y_train)
reg4.fit(x_train, y_train)
reg5.fit(x_train, y_train)

pred_train = reg.predict(x_train)
pred_train2 = reg2.predict(x_train)
pred_train3 = reg3.predict(x_train)
pred_train4 = reg4.predict(x_train)
pred_train5 = reg5.predict(x_train)

pred_val = reg.predict(x_test)
pred_val2 = reg2.predict(x_test)
pred_val3 = reg3.predict(x_test)
pred_val4 = reg4.predict(x_test)
pred_val5 = reg5.predict(x_test)

mae_train = mean_absolute_error(y_train, pred_train)
mae_val = mean_absolute_error(y_test, pred_val)
mae_train2 = mean_absolute_error(y_train, pred_train2)
mae_val2 = mean_absolute_error(y_test, pred_val2)
mae_train3 = mean_absolute_error(y_train, pred_train3)
mae_val3 = mean_absolute_error(y_test, pred_val3)
mae_train4 = mean_absolute_error(y_train, pred_train4)
mae_val4 = mean_absolute_error(y_test, pred_val4)
mae_train5 = mean_absolute_error(y_train, pred_train5)
mae_val5 = mean_absolute_error(y_test, pred_val5)


print('1. Linear Regression\t, train=%.4f, val=%.4f' % (mae_train, mae_val))
print('2. Lasso\t\t, train=%.4f, val=%.4f' % (mae_train2, mae_val2))
print('3. Ridge\t\t, train=%.4f, val=%.4f' % (mae_train3, mae_val3))
print('4. XGBoost\t\t, train=%.4f, val=%.4f' % (mae_train4, mae_val4))
print('5. LightGBM\t\t, train=%.4f, val=%.4f' % (mae_train5, mae_val5))

In [ ]:
# prediction!
result = reg.predict(x_test)
result2 = reg2.predict(x_test)
result3 = reg3.predict(x_test)
result4 = reg4.predict(x_test)
result5 = reg5.predict(x_test)

In [ ]:
# Summary!
print('---------- Linear Regression ---------')
print('MsE in training: %.4f' % mean_absolute_error(y_test, result))

print('---------- Lasso ---------')
print('MsE in training: %.4f' % mean_absolute_error(y_test, result2))

print('---------- Ridge ---------')
print('MsE in training: %.4f' % mean_absolute_error(y_test, result3))

print('---------- XGBoost ---------')
print('MsE in training: %.4f' % mean_absolute_error(y_test, result4))

print('---------- LightGBM ---------')
print('MsE in training: %.4f' % mean_absolute_error(y_test, result5))

In [ ]:
# Before Drop / After Drop => check

** Before drop colums **

---------- Linear Regression ---------

MSE in training: 0.0926

---------- Lasso ---------

MSE in training: 0.1209

---------- Ridge ---------

MSE in training: 0.0926

---------- XGBoost ---------

MSE in training: 0.0694

---------- LightGBM ---------

MSE in training: 0.0603

---

** After drop columns **

---------- Linear Regression ---------

MSE in training: 0.0926

---------- Lasso ---------

MSE in training: 0.1209

---------- Ridge ---------

MSE in training: 0.0926

---------- XGBoost ---------

MSE in training: 0.0694

---------- LightGBM ---------

MSE in training: 0.0603

In [ ]:
# No change checked.